# Laboratorio 8

In [147]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

data = pd.read_csv('titanic_MD.csv')

In [148]:
print('Titanic_MD dataset shape:', data.shape)
data.head()

Titanic_MD dataset shape: (183, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",?,38.0,1.0,0.0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0.0,0.0,17463,51.8625,E46,S
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,NaN,1.0,NaN,PP 9549,16.7000,G6,S
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,NaN,0.0,113783,26.5500,C103,S


In [149]:
data.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,158.000000,180.000000,171.000000,175.000000
mean,455.366120,0.672131,1.191257,35.692532,0.461111,0.461988,78.959191
std,247.052476,0.470725,0.515187,15.640858,0.646122,0.753435,77.026328
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,35.500000,0.000000,0.000000,56.929200
75%,676.000000,1.000000,1.000000,48.000000,1.000000,1.000000,90.539600
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


# 1. Missing Data

In [150]:
data.notna()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,False,True,False,True,True,True,True
4,True,True,True,True,True,True,False,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
178,True,True,True,True,True,True,True,True,True,False,True,True
179,True,True,True,True,True,False,True,True,True,True,True,True
180,True,True,True,True,True,True,True,False,True,True,True,False
181,True,True,True,True,True,True,True,True,True,True,True,True


##### Función "isna()" contiene la palabra "NA" y los vacīos

In [151]:
data.isna().sum()

PassengerId     0
Survived        0
Pclass          0
Name            0
Sex             0
Age            25
SibSp           3
Parch          12
Ticket          0
Fare            8
Cabin           0
Embarked       12
dtype: int64

In [152]:
cols = []
val = []
for col in data.select_dtypes(include='object').columns:
    cols.append(col)
    val.append(data[col].str.contains(r'\?').sum())
pd.DataFrame({
    'cols':cols,
    'val':val
})

,cols,val
0,Name,0
1,Sex,51
2,Ticket,0
3,Cabin,0
4,Embarked,0


In [153]:

data.replace(r'\?', np.nan, regex = True, inplace = True)
data.isna().sum()

PassengerId     0
Survived        0
Pclass          0
Name            0
Sex            51
Age            25
SibSp           3
Parch          12
Ticket          0
Fare            8
Cabin           0
Embarked       12
dtype: int64

# 2. Para cada columna especificar que tipo de modelo se utilizará y qué valores se le darán a todos los missing values. 

##### Sex: Modelo de regresión lineal
##### Age: Imputación por media
##### SibSp: Imputación por moda
##### Parch: Imputación por moda
##### Fare: Imputación por media
##### Embarked: Vecino más cercano

# 3. Reporte de qué filas están completas 

In [173]:
#Columnas NA

In [174]:
data.isna().any()


PassengerId    False
Survived       False
Pclass         False
Name           False
Sex             True
Age             True
SibSp           True
Parch           True
Ticket         False
Fare            True
Cabin          False
Embarked        True
dtype: bool

# funciones para limpiar Na¶ 

### Pairwise Deletion

In [175]:

def pairwise_deletion(dataframe, columns): #columns es un arreglo
    dataframe = dataframe.dropna(subset=columns)
    return dataframe

### Imputación Media

In [176]:
def imputation_mean(dataframe, column_name=None):
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    if column_name == None:
        for col in dataframe:
            dataframe[col] = imp.fit_transform(dataframe[[col]])
    else:
        dataframe[[column_name]] = imp.fit_transform(dataframe[[column_name]])   
    return dataframe

### Imputación Moda

In [177]:
def imputation_mode(dataframe, column_name=None):    
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    if column_name == None:
        for col in dataframe:
            dataframe[col] = imp.fit_transform(dataframe[[col]])
    else:
        dataframe[[column_name]] = imp.fit_transform(dataframe[[column_name]])        
    return dataframe

### Imputación Mediana

In [178]:
def imputation_median(dataframe, column_name=None):    
    imp = SimpleImputer(missing_values=np.nan, strategy='median')
    if column_name == None:
        for col in dataframe:
            dataframe[col] = imp.fit_transform(dataframe[[col]])
    else:
        dataframe[[column_name]] = imp.fit_transform(dataframe[[column_name]])   
    return dataframe


### Imputación Sectorizada

In [179]:
def imputation_sectorized(dataframe, column_name=None, fill=None):
    imp = SimpleImputer(missing_values=np.nan, strategy='median', fill_value=fill)
    if column_name == None:
        for col in dataframe:
            dataframe[col] = imp.fit_transform(dataframe[[col]])
    else:
        dataframe[[column_name]] = imp.fit_transform(dataframe[[column_name]])  
    return dataframe

In [180]:
      
def eliminateAllNa(dataframe):
    cols = []
    val = []
    for col in dataframe.select_dtypes(include='object').columns:
        cols.append(col)
        val.append(dataframe[col].str.contains(r'\?').sum())
    pd.DataFrame({
        'cols':cols,
        'val':val
    })
    
    dataframe.replace(r'\?', np.nan, regex = True, inplace = True)
    
    return dataframe

# 4. Utilizar los siguientes métodos para cada columna que contiene missing values:

In [181]:
print('Titanic_MD dataset shape:', data.shape)
data.describe()

Titanic_MD dataset shape: (183, 12)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,158.000000,180.000000,171.000000,175.000000
mean,455.366120,0.672131,1.191257,35.692532,0.461111,0.461988,78.959191
std,247.052476,0.470725,0.515187,15.640858,0.646122,0.753435,77.026328
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,35.500000,0.000000,0.000000,56.929200
75%,676.000000,1.000000,1.000000,48.000000,1.000000,1.000000,90.539600
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


# PairWise Deletion

In [182]:
data.corr()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,0.148495,-0.089136,-0.048190,-0.088806,-0.062083,0.022261
Survived,0.148495,1.000000,-0.034542,-0.257703,0.113987,-0.003365,0.119311
Pclass,-0.089136,-0.034542,1.000000,-0.297872,-0.102294,0.041969,-0.304438
Age,-0.048190,-0.257703,-0.297872,1.000000,-0.087951,-0.279548,-0.130979
SibSp,-0.088806,0.113987,-0.102294,-0.087951,1.000000,0.255152,0.299061
Parch,-0.062083,-0.003365,0.041969,-0.279548,0.255152,1.000000,0.381445
Fare,0.022261,0.119311,-0.304438,-0.130979,0.299061,0.381445,1.000000


In [183]:
titanic_pairwise = pairwise_deletion(data, ["Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"])
print('Titanic_MD PairWise Deletion shape:', titanic_pairwise.shape)
titanic_pairwise.describe()

Titanic_MD PairWise Deletion shape: (100, 12)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000
mean,467.120000,0.620000,1.160000,36.124200,0.44000,0.480000,74.445126
std,230.014944,0.487832,0.465366,15.123837,0.53786,0.731402,65.975260
min,4.000000,0.000000,1.000000,0.920000,0.00000,0.000000,0.000000
25%,316.750000,0.000000,1.000000,24.000000,0.00000,0.000000,31.990600
50%,480.500000,1.000000,1.000000,36.000000,0.00000,0.000000,59.979200
75%,649.500000,1.000000,1.000000,48.250000,1.00000,1.000000,90.000000
max,868.000000,1.000000,3.000000,71.000000,2.00000,2.000000,512.329200


# Imputacion usando la media¶


In [184]:
data = pd.read_csv('titanic_MD.csv')
data = eliminateAllNa(data)
missings = ["Age", "SibSp", "Parch", "Fare"]
for col in missings:
    titanic_mean = imputation_mean(data, col)
print('Titanic_MD Mean Imputation shape:', titanic_mean.shape)
titanic_mean.describe()

Titanic_MD Mean Imputation shape: (183, 12)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,35.692532,0.461111,0.461988,78.959191
std,247.052476,0.470725,0.515187,14.526959,0.640775,0.728173,75.314418
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,25.000000,0.000000,0.000000,30.000000
50%,457.000000,1.000000,1.000000,35.692532,0.000000,0.000000,61.175000
75%,676.000000,1.000000,1.000000,46.500000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


# Imputacion usando la moda¶

In [185]:
data = pd.read_csv('titanic_MD.csv')
data = eliminateAllNa(data)
missings = ["Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
for col in missings:
    titanic_mode = imputation_mode(data, col)
print('Titanic_MD Mode Imputation shape:', titanic_mode.shape)
titanic_mode.describe()

Titanic_MD Mode Imputation shape: (183, 12)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,34.095191,0.453552,0.431694,76.668079
std,247.052476,0.470725,0.515187,15.074700,0.643459,0.737147,76.077057
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,26.550000
50%,457.000000,1.000000,1.000000,31.000000,0.000000,0.000000,55.441700
75%,676.000000,1.000000,1.000000,46.500000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


 ## Imputacion usando mediana

In [186]:
data = pd.read_csv('titanic_MD.csv')
data = eliminateAllNa(data)
missings = ["Age", "SibSp", "Parch", "Fare"]
for col in missings:
    titanic_median = imputation_median(data, col)
print('Titanic_MD Mode Imputation shape:', titanic_median.shape)
titanic_median.describe()

Titanic_MD Mode Imputation shape: (183, 12)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.00000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,35.66623,0.453552,0.431694,77.996131
std,247.052476,0.470725,0.515187,14.52711,0.643459,0.737147,75.449730
min,2.000000,0.000000,1.000000,0.92000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,25.00000,0.000000,0.000000,30.000000
50%,457.000000,1.000000,1.000000,35.50000,0.000000,0.000000,56.929200
75%,676.000000,1.000000,1.000000,46.50000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.00000,3.000000,4.000000,512.329200


# Imputacion Sectorizada

In [187]:
data = pd.read_csv('titanic_MD.csv')
data = eliminateAllNa(data)
missings = ["Age", "SibSp", "Parch", "Fare"]
for col in missings:
    titanic_sectorized = imputation_sectorized(data, col)
print('Titanic_MD Sectorized Imputation shape:', titanic_sectorized.shape)
titanic_sectorized.describe()

Titanic_MD Sectorized Imputation shape: (183, 12)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.00000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,35.66623,0.453552,0.431694,77.996131
std,247.052476,0.470725,0.515187,14.52711,0.643459,0.737147,75.449730
min,2.000000,0.000000,1.000000,0.92000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,25.00000,0.000000,0.000000,30.000000
50%,457.000000,1.000000,1.000000,35.50000,0.000000,0.000000,56.929200
75%,676.000000,1.000000,1.000000,46.50000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.00000,3.000000,4.000000,512.329200


 # Standard Deviation¶


In [169]:
data = pd.read_csv('titanic_MD.csv')
data = eliminateAllNa(data)
missings = ["Age", "SibSp", "Parch", "Fare"]
for col in missings:
    titanic_standard = outliers_standard(data, col, 2)
print('Titanic_MD Outliers Standard shape:', titanic_standard.shape)
titanic_standard.describe()

Titanic_MD Outliers Standard shape: (0, 12)


<ipython-input-51-b7d7cd6a3f87>:50: RuntimeWarning: invalid value encountered in less
  dataframe = dataframe[(np.abs(stats.zscore(dataframe[[column]])) < zscore).all(axis=1)]


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 5. Al comparar los métodos del inciso 4 contra “titanic.csv”, ¿Qué método (para cada columna) se acerca más a la realidad y por qué? 

#### La imputación mejor acertada para las columnas de Fare y Age considero que es la imputación por la media ya que los valores estan bien distribuidos para que la imputacion de un resultado parcial y el analisis no se vea afectado. Ahora para las columnas de Parch, Embarked y SibSp la imputacion por moda es la mejor opcion. No es recomendable utilizar la media para las columnas de Parch y SibSp porque no hay cantidad de valores suficientes por lo que añadir varios valores de la media puede ser considerado imparcial. Con la columna Sex lo mas recomendable seria el uso de una regresión lineal debido a la cantidad de NA. 


# 6. Conclusiones

#### - Al Df le hace falta estandarizar sus NA
#### -Imputacion por la mediana no es un método comunmente utilizado
#### - Es recomendable imputar los datos en lugar de eliminarlos
#### - Unicamente se deben eliminar filas que contengan NA si la imputación de datos no fuera posible
#### - No existen variedad de opciones de imputacion para variables categoricas, pero si para numericas
#### - Determinar los NA de género es dificil por la falta del resto de información

# Parte 2

# 1. Luego del pre-procesamiento de la data con Missing Values, normalice las columnasnuméricas por los métodos: a. Standarization b. MinMaxScaling c. MaxAbsScaler

# Standarization

In [170]:
data = pd.read_csv('titanic_MD.csv')
data = eliminateAllNa(data)

scaler = StandardScaler()
df_z = data.copy()
for col in df_z.select_dtypes(include=['float', 'int']).columns:
    df_z[col+'_z'] = scaler.fit_transform(df_z[[col]])
    
df_z.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,PassengerId_z,Survived_z,Pclass_z,Age_z,SibSp_z,Parch_z,Fare_z
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",NaN,38.0,1.0,0.0,PC 17599,71.2833,C85,C,-1.840135,0.698430,-0.372256,0.147997,0.836362,-0.614977,-0.099939
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S,-1.832017,0.698430,-0.372256,-0.044418,0.836362,-0.614977,-0.336682
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0.0,0.0,17463,51.8625,E46,S,-1.819841,-1.431782,-0.372256,1.174212,-0.715650,-0.614977,-0.352794
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,NaN,1.0,NaN,PP 9549,16.7000,G6,S,-1.803606,0.698430,3.520480,NaN,0.836362,NaN,-0.810604
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,NaN,0.0,113783,26.5500,C103,S,-1.799547,0.698430,-0.372256,1.430765,NaN,-0.614977,-0.682359


# MinMaxScaling

In [171]:
data = pd.read_csv('titanic_MD.csv')
data = eliminateAllNa(data)

scaler = MinMaxScaler()
df_norm = data.copy()
for col in df_norm.select_dtypes(include=['float', 'int']).columns:
    df_norm[col+'_norm'] = scaler.fit_transform(df_norm[[col]])    
df_norm.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,PassengerId_norm,Survived_norm,Pclass_norm,Age_norm,SibSp_norm,Parch_norm,Fare_norm
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",NaN,38.0,1.0,0.0,PC 17599,71.2833,C85,C,0.000000,1.0,0.0,0.468892,0.333333,0.0,0.139136
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S,0.002252,1.0,0.0,0.430956,0.333333,0.0,0.103644
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0.0,0.0,17463,51.8625,E46,S,0.005631,0.0,0.0,0.671219,0.000000,0.0,0.101229
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,NaN,1.0,NaN,PP 9549,16.7000,G6,S,0.010135,1.0,1.0,NaN,0.333333,NaN,0.032596
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,NaN,0.0,113783,26.5500,C103,S,0.011261,1.0,0.0,0.721801,NaN,0.0,0.051822


## MaxAbsScaler

In [172]:
data = pd.read_csv('titanic_MD.csv')
data = eliminateAllNa(data)

scaler = MaxAbsScaler()
df_abs = data.copy()
for col in df_abs.select_dtypes(include=['float', 'int']).columns:
    df_abs[col+'_norm'] = scaler.fit_transform(df_abs[[col]])    
df_abs.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,PassengerId_norm,Survived_norm,Pclass_norm,Age_norm,SibSp_norm,Parch_norm,Fare_norm
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",NaN,38.0,1.0,0.0,PC 17599,71.2833,C85,C,0.002247,1.0,0.333333,0.4750,0.333333,0.0,0.139136
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S,0.004494,1.0,0.333333,0.4375,0.333333,0.0,0.103644
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0.0,0.0,17463,51.8625,E46,S,0.007865,0.0,0.333333,0.6750,0.000000,0.0,0.101229
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,NaN,1.0,NaN,PP 9549,16.7000,G6,S,0.012360,1.0,1.000000,NaN,0.333333,NaN,0.032596
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,NaN,0.0,113783,26.5500,C103,S,0.013483,1.0,0.333333,0.7250,NaN,0.0,0.051822
